# TensorFlow Tutorial

In [1]:
## note we need to make sure these environment variables are set properly for tensorflow to properly make use of the GPU
# should be: /app/apps/rhel8/jupyter/2023-04/lib/python3.10/site-packages/nvidia/cudnn
!echo $CUDNN_PATH
# should be: /usr/local/cuda/lib64:/app/apps/rhel8/jupyter/2023-04/lib/python3.10/site-packages/nvidia/cudnn/lib:/app/apps/rhel8/jupyter/2023-04/lib/python3.10/site-packages/tensorrt_libs::
!echo $LD_LIBRARY_PATH
# must include: /app/apps/rhel8/jupyter/2023-04/bin:/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/opt/jupyterhub/bin:/common/bin:/usr/local/cuda/bin:/usr/localsys/bin:/usr/share/Modules/bin
!echo $PATH

# test to see if changed thing in config fixes git push

/app/apps/rhel8/jupyter/2023-04/lib/python3.10/site-packages/nvidia/cudnn
/usr/local/cuda/lib64:/app/apps/rhel8/jupyter/2023-04/lib/python3.10/site-packages/nvidia/cudnn/lib:/app/apps/rhel8/jupyter/2023-04/lib/python3.10/site-packages/tensorrt_libs::
/app/apps/rhel8/jupyter/2023-04/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/opt/jupyterhub/bin:/common/bin:/usr/local/cuda/bin:/usr/localsys/bin:/usr/share/Modules/bin:/bin


In [2]:
import tensorflow as tf

2023-09-08 22:50:13.715067: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-08 22:50:13.773161: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
v1=set("/app/apps/rhel8/jupyter/2023-04/lib/python3.10/site-packages/nvidia/cudnn/lib:/app/apps/rhel8/jupyter/2023-04/lib/python3.10/site-packages/tensorrt_libs:/app/apps/rhel8/python-anaconda3/2023-04/lib:/app/apps/rhel8/jupyter/2023-04/lib/python3.10/site-packages/nvidia/cudnn/lib:/home/craut/miniconda3/lib:/usr/local/cuda/lib64::".split(":"))
v2=set("/usr/local/cuda/lib64:/usr/local/cuda/lib64:/app/apps/rhel8/jupyter/2023-04/lib/python3.10/site-packages/nvidia/cudnn/lib:/app/apps/rhel8/jupyter/2023-04/lib/python3.10/site-packages/tensorrt_libs::".split(":"))
v1-v2

{'/app/apps/rhel8/python-anaconda3/2023-04/lib', '/home/craut/miniconda3/lib'}

In [ ]:
# proposed test
LD_LIBRARY_PATH=/usr/local/cuda/lib64:/usr/local/cuda/lib64:/app/apps/rhel8/jupyter/2023-04/lib/python3.10/site-packages/nvidia/cudnn/lib:/app/apps/rhel8/jupyter/2023-04/lib/python3.10/site-packages/tensorrt_libs:/app/apps/rhel8/python-anaconda3/2023-04/lib::

In [3]:
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.13.0


In [4]:
from tensorflow import keras
from tensorflow.keras import layers, models, datasets
import time
from tensorflow.python.client import device_lib

In [5]:
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU' or x.device_type == 'CPU']
get_available_devices()

2023-09-08 22:50:18.755170: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


['/device:CPU:0']

In [6]:
# Download and prepare the CIFAR10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Create the convolutional base
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Add Dense layers on top
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

# Compile and train the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
# Fit the model on GPU
start_time = time.time()
history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels))
end_time = time.time()
print("Time to fit model on GPU: {:.2f} seconds".format(end_time - start_time))

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test_images, test_labels)
print("test loss, test acc:", results)

2023-09-08 22:50:19.985368: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/10
1563/1563 [==============================] - 10s 5ms/step - loss: 1.5927 - accuracy: 0.4097 - val_loss: 1.2883 - val_accuracy: 0.5299
Epoch 2/10
1563/1563 [==============================] - 7s 5ms/step - loss: 1.2276 - accuracy: 0.5580 - val_loss: 1.1462 - val_accuracy: 0.5887
Epoch 3/10
1563/1563 [==============================] - 7s 5ms/step - loss: 1.0681 - accuracy: 0.6219 - val_loss: 1.0581 - val_accuracy: 0.6283
Epoch 4/10
1563/1563 [==============================] - 7s 5ms/step - loss: 0.9616 - accuracy: 0.6624 - val_loss: 1.0032 - val_accuracy: 0.6569
Epoch 5/10
1563/1563 [==============================] - 9s 5ms/step - loss: 0.8759 - accuracy: 0.6921 - val_loss: 0.9154 - val_accuracy: 0.6831
Epoch 6/10
1563/1563 [==============================] - 7s 5ms/step - loss: 0.8084 - accuracy: 0.7168 - val_loss: 0.9028 - val_accuracy: 0.6882
Epoch 7/10
1563/1563 [==============================] - 7s 5ms/step - loss: 0.7605 - accuracy: 0.7342 - val_loss: 0.8793 - val_accuracy

In [7]:
# code that I wrote for the PDFF imputation project using a baseline_mlp model
!cd /home/craut/wkspce/craut_lfs/MRI_PDFF_imputation/fit_models/ && time python evaluate_model.py -dd ../cut_data_again/real_data/ -od output2 -m baseline_mlp -tr -te -i simple

Num GPUs Available:  0
### TRAINING ###
(34634, 248) (34634,)
Fitting Model on All Data
Epoch 1/100
217/217 [==============================] - 2s 4ms/step - loss: 25.3892 - root_mean_squared_error: 4.7777 - mean_absolute_error: 3.0563 - val_loss: 15.6401 - val_root_mean_squared_error: 3.6439 - val_mean_absolute_error: 2.2754
Epoch 2/100
217/217 [==============================] - 1s 3ms/step - loss: 18.4416 - root_mean_squared_error: 4.0246 - mean_absolute_error: 2.4646 - val_loss: 17.9230 - val_root_mean_squared_error: 3.9735 - val_mean_absolute_error: 2.3135
Epoch 3/100
217/217 [==============================] - 1s 3ms/step - loss: 16.9820 - root_mean_squared_error: 3.8656 - mean_absolute_error: 2.3692 - val_loss: 16.5215 - val_root_mean_squared_error: 3.8177 - val_mean_absolute_error: 2.9222
Epoch 4/100
217/217 [==============================] - 1s 3ms/step - loss: 16.1200 - root_mean_squared_error: 3.7772 - mean_absolute_error: 2.3055 - val_loss: 27.2791 - val_root_mean_squared_erro